# Analisi delle Serie Temporali — Qualità dell'Aria e Ricoveri Respiratori

**Dataset:** `air_quality_health_dataset.csv`  
**Regione:** East (635 osservazioni giornaliere → 105 settimane dopo resample)

### Pipeline di analisi
1. Caricamento e preprocessing
2. Filtraggio regione East + resample settimanale
3. EDA: statistiche per anno/stagione, scatter con lag, correlazioni crociate
4. Decomposizione stagionale additiva
5. Test di stazionarietà ADF
6. Analisi ACF/PACF
7. Selezione automatica parametri (`auto_arima`, `d=None`, `D=None`)
8. Train/Test split 80/20 su `df_weekly`
9. Fit e confronto di tre modelli
10. Tabella comparativa finale e conclusioni

## 1. Importazione librerie

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib.dates as mdates
import warnings
warnings.filterwarnings('ignore')

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller, ccf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.stats.diagnostic import acorr_ljungbox
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import jarque_bera, shapiro
import pmdarima as pm

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

## 2. Caricamento e Ispezione del Dataset

In [ ]:
df = pd.read_csv('air_quality_health_dataset.csv')
df = df[['date', 'region', 'PM2.5', 'NO2', 'O3', 'temperature',
         'humidity', 'wind_speed', 'respiratory_admissions']]
df.head()

In [ ]:
df.tail()

In [ ]:
df.info()
print("\nValori nulli:")
print(df.isnull().sum())
print("\nDuplicati:", df.duplicated().sum())
print("\nRegioni presenti:")
print(df['region'].value_counts())

In [ ]:
df.describe()

In [ ]:
# Verifica duplicati
df[df.duplicated()]

## 3. Filtraggio Regione East e Resample Settimanale

Si lavora su una sola regione per isolare una serie temporale pulita e uniforme.
- `.copy()` evita il `SettingWithCopyWarning`
- `index.freq = 'W'` dopo il resample evita il `ValueWarning` di statsmodels
- I ricoveri vengono **sommati** (conteggio settimanale); le variabili ambientali vengono **mediate**

In [ ]:
# FIX: .copy() per evitare SettingWithCopyWarning
df_east = df[df['region'] == 'East'].copy()
df_east['date'] = pd.to_datetime(df_east['date'])
df_east.set_index('date', inplace=True)
df_east.sort_index(inplace=True)

df_east.head()

In [ ]:
aggregation_logic = {
    'PM2.5': 'mean',
    'NO2': 'mean',
    'O3': 'mean',
    'temperature': 'mean',
    'humidity': 'mean',
    'wind_speed': 'mean',
    'respiratory_admissions': 'sum'   # somma settimanale dei ricoveri
}

df_weekly = df_east.resample('W').agg(aggregation_logic)
df_weekly.index.freq = 'W'   # FIX: evita ValueWarning di statsmodels durante il forecasting

print(f"Serie settimanale: {len(df_weekly)} settimane")
print(f"Periodo: {df_weekly.index[0].date()} → {df_weekly.index[-1].date()}")
print(df_weekly.head())

In [ ]:
df_weekly.isnull().sum()

## 4. Analisi Esplorativa (EDA)

### 4.1 Statistiche Descrittive per Anno e Stagione

In [ ]:
df_weekly['year'] = df_weekly.index.year
df_weekly['month'] = df_weekly.index.month

def get_season(month):
    if month in [12, 1, 2]: return 'Winter'
    elif month in [3, 4, 5]: return 'Spring'
    elif month in [6, 7, 8]: return 'Summer'
    else: return 'Autumn'

df_weekly['season'] = df_weekly['month'].apply(get_season)

In [ ]:
cols_to_analyze = ['PM2.5', 'NO2', 'O3', 'temperature', 'humidity', 'wind_speed', 'respiratory_admissions']

print("=== Statistiche descrittive per ANNO ===")
yearly_stats = df_weekly.groupby('year')[cols_to_analyze].agg(['mean', 'var', 'min', 'max'])
yearly_stats_vertical = yearly_stats.stack(level=0).round(2)
print(yearly_stats_vertical)
print("\nNOTA: 2022 ha una sola settimana di dati → var=NaN e statistiche non rappresentative.")

In [ ]:
print("\n=== Statistiche descrittive per STAGIONE ===")
seasonal_stats = df_weekly.groupby('season')[cols_to_analyze].agg(['mean', 'var', 'min', 'max'])
seasonal_stats_vertical = seasonal_stats.stack(level=0).round(2)
seasonal_stats_vertical.index.names = ['Stagione', 'Variabile']
print(seasonal_stats_vertical)

**Media nel tempo:** i livelli medi di PM2.5 e temperatura risultano stabili tra 2020 e 2021 (PM2.5 ≈ 59, temperatura ≈ 25°C), senza trend marcati. I ricoveri respiratori mostrano una media superiore nel 2020 (65.04) rispetto al 2021 (55.40), suggerendo una leggera tendenza decrescente.

**Varianza:** il PM2.5 mostra varianza più che doppia nel 2021 (62.08 vs 28.48 del 2020), indicando episodi più irregolari di inquinamento. La varianza dei ricoveri rimane elevata in entrambi gli anni.

**Stagionalità:** i ricoveri sono più alti in estate (63.65) e autunno (59.96), mentre la primavera ha i valori più bassi (53.37). Questo pattern è coerente con effetti di ondate di calore piuttosto che con la classica stagionalità invernale.

**Conclusione:** le serie presentano variabilità non costante nel tempo e una componente stagionale moderata, che rendono necessaria un'analisi formale di stazionarietà prima della modellizzazione.

### 4.2 Scatter Plot: PM2.5 e Temperatura vs Ricoveri (Lag 0, 1, 5/6)

In [ ]:
print("─"*80)
print("SCATTER PLOTS: PM2.5 vs Ricoveri (Lag 0, 1, 5)")
print("─"*80)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(22, 6))

for ax, lag, color, xlabel in zip(
    [ax1, ax2, ax3],
    [0, 1, 5],
    ['steelblue', 'tomato', 'seagreen'],
    ['PM2.5 (t)', 'PM2.5 (t-1 sett.)', 'PM2.5 (t-5 sett.)']
):
    x = df_weekly['PM2.5'].shift(lag)
    sns.scatterplot(x=x, y=df_weekly['respiratory_admissions'],
                    alpha=0.6, ax=ax, s=60, color=color,
                    edgecolor='black', linewidth=0.5)
    # Linea di tendenza
    valid = pd.concat([x, df_weekly['respiratory_admissions']], axis=1).dropna()
    if len(valid) > 1:
        z = np.polyfit(valid.iloc[:, 0], valid.iloc[:, 1], 1)
        xr = np.linspace(valid.iloc[:, 0].min(), valid.iloc[:, 0].max(), 100)
        ax.plot(xr, np.poly1d(z)(xr), 'k--', linewidth=1.5, alpha=0.6)
    ax.set_title(f'PM2.5 Lag {lag} vs Ricoveri', fontsize=12, fontweight='bold')
    ax.set_xlabel(xlabel, fontsize=11)
    ax.set_ylabel('Ricoveri Settimanali', fontsize=11)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
print("─"*80)
print("SCATTER PLOTS: Temperatura vs Ricoveri (Lag 0, 1, 6)")
print("─"*80)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(22, 6))

for ax, lag, color, xlabel in zip(
    [ax1, ax2, ax3],
    [0, 1, 6],
    ['steelblue', 'tomato', 'seagreen'],
    ['Temp (t)', 'Temp (t-1 sett.)', 'Temp (t-6 sett.)']
):
    x = df_weekly['temperature'].shift(lag)
    sns.scatterplot(x=x, y=df_weekly['respiratory_admissions'],
                    alpha=0.6, ax=ax, s=60, color=color,
                    edgecolor='black', linewidth=0.5)
    valid = pd.concat([x, df_weekly['respiratory_admissions']], axis=1).dropna()
    if len(valid) > 1:
        z = np.polyfit(valid.iloc[:, 0], valid.iloc[:, 1], 1)
        xr = np.linspace(valid.iloc[:, 0].min(), valid.iloc[:, 0].max(), 100)
        ax.plot(xr, np.poly1d(z)(xr), 'k--', linewidth=1.5, alpha=0.6)
    ax.set_title(f'Temperatura Lag {lag} vs Ricoveri', fontsize=12, fontweight='bold')
    ax.set_xlabel(xlabel, fontsize=11)
    ax.set_ylabel('Ricoveri Settimanali', fontsize=11)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 4.3 Correlazione Incrociata (CCF)

In [ ]:
conf_band = 1.96 / np.sqrt(len(df_weekly))

fig, axes = plt.subplots(1, 2, figsize=(18, 5))

for ax, var, title, color in zip(
    axes,
    ['PM2.5', 'temperature'],
    ['CCF: PM2.5 → Ricoveri Respiratori', 'CCF: Temperatura → Ricoveri Respiratori'],
    ['steelblue', 'tomato']
):
    lags_n = 15
    cc = ccf(df_weekly['respiratory_admissions'].dropna(),
             df_weekly[var].dropna())[:lags_n+1]
    ax.stem(range(lags_n+1), cc, linefmt=color, markerfmt='o', basefmt='grey')
    ax.axhline( conf_band, color='red', linestyle='--', alpha=0.7, linewidth=1.5, label='95% CI')
    ax.axhline(-conf_band, color='red', linestyle='--', alpha=0.7, linewidth=1.5)
    ax.set_title(title, fontsize=13, fontweight='bold')
    ax.set_xlabel('Lag (Settimane)', fontsize=11)
    ax.set_ylabel('Coefficiente di Correlazione', fontsize=11)
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
print("Valori che superano la banda rossa tratteggiata sono statisticamente significativi (p < 0.05).")

L'analisi esplorativa non evidenzia una relazione forte tra PM2.5 e ricoveri respiratori nella stessa settimana. Introducendo ritardi temporali emerge un segnale più coerente a lag 1-2 settimane, suggerendo un possibile effetto ritardato dell'esposizione al PM2.5. La temperatura mostra una correlazione debole e non sistematica. Nel complesso, entrambi gli effetti richiedono modelli dinamici con lag espliciti.

### 4.4 Distribuzione dei Ricoveri: Grezzi vs Differenziati

In [ ]:
print("\n" + "─"*80)
print("DISTRIBUZIONE RICOVERI: Grezzi vs Differenziati")
print("─"*80 + "\n")

plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
sns.histplot(df_weekly['respiratory_admissions'], kde=True, color='steelblue', bins=30)
plt.title('Distribuzione Ricoveri (Grezzi)', fontsize=13, fontweight='bold')
plt.xlabel('Ricoveri Settimanali', fontsize=11)
plt.ylabel('Frequenza', fontsize=11)
plt.grid(True, alpha=0.3, axis='y')

plt.subplot(1, 2, 2)
diff_admissions = df_weekly['respiratory_admissions'].diff().dropna()
sns.histplot(diff_admissions, kde=True, color='seagreen', bins=30)
plt.title('Distribuzione Variazioni (Differenze)', fontsize=13, fontweight='bold')
plt.xlabel('Δ Ricoveri', fontsize=11)
plt.ylabel('Frequenza', fontsize=11)
plt.axvline(0, color='red', linestyle='--', linewidth=2, alpha=0.7)
plt.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()
print("La distribuzione delle differenze è più simmetrica e centrata su zero → la serie può essere resa stazionaria.")

Gli istogrammi mostrano che i ricoveri grezzi hanno una distribuzione asimmetrica con coda destra, mentre le differenze presentano una distribuzione molto più simmetrica e centrata sullo zero. Questo indica che una differenziazione ordinaria (d=1) è sufficiente a rimuovere la non-stazionarietà in media.

### 4.5 Serie Temporale dei Ricoveri

In [ ]:
fig, ax = plt.subplots(figsize=(20, 7))
ax.plot(df_weekly.index, df_weekly['respiratory_admissions'],
        linewidth=2, color='#2E86AB', label='Ricoveri settimanali')

# Media mobile a 4 settimane per evidenziare il trend locale
ma4 = df_weekly['respiratory_admissions'].rolling(4, center=True).mean()
ax.plot(df_weekly.index, ma4, color='tomato', linewidth=2.5,
        linestyle='--', alpha=0.85, label='Media mobile 4 sett.')

ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
plt.xticks(rotation=45, fontsize=11)
ax.set_title('Ricoveri Respiratori Settimanali – East Region', fontsize=16, fontweight='bold')
ax.set_xlabel('Data')
ax.set_ylabel('Ricoveri Settimanali')
ax.legend(fontsize=12)
ax.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

La serie mostra oscillazioni irregolari con variabilità elevata. La media mobile a 4 settimane evidenzia una tendenza decrescente nel 2020-2021, con media non costante nel tempo e varianza non uniforme. Questi elementi verranno verificati formalmente con il test ADF e la decomposizione stagionale.

## 5. Decomposizione Stagionale Additiva

La decomposizione additiva separa la serie in: **Trend + Stagionalità (period=52) + Residui**.  
`extrapolate_trend='freq'` gestisce i bordi della serie dove il filtro a media mobile non ha dati sufficienti.

In [ ]:
print("="*80)
print("FASE 5: DECOMPOSIZIONE STAGIONALE")
print("="*80)

series_for_decomp = df_weekly['respiratory_admissions'].dropna()

decomposition = seasonal_decompose(
    series_for_decomp,
    model='additive',
    period=52,
    extrapolate_trend='freq'
)

fig, axes = plt.subplots(4, 1, figsize=(20, 14), sharex=True)
components = [
    (decomposition.observed,  'Serie Osservata',                            '#2E86AB'),
    (decomposition.trend,     'Trend (Evoluzione Lungo Periodo)',            '#A23B72'),
    (decomposition.seasonal,  'Componente Stagionale (Periodicità Annuale)', '#F18F01'),
    (decomposition.resid,     'Residui (Rumore/Irregolarità)',               '#6A994E')
]

for i, (data, title, color) in enumerate(components):
    axes[i].plot(data, color=color, linewidth=2, alpha=0.9)
    axes[i].set_title(title, fontsize=13, fontweight='bold', pad=10)
    axes[i].set_ylabel('Ricoveri', fontsize=11)
    axes[i].grid(True, linestyle='--', alpha=0.4)

axes[3].set_xlabel('Data', fontsize=11)
plt.tight_layout()
plt.show()

seasonal_strength = 1 - (decomposition.resid.var() /
                         (decomposition.seasonal + decomposition.resid).var())
print(f"\nForza componente stagionale: {seasonal_strength:.3f}  (0=assente, 1=molto forte)")
print("Con soli ~2 anni di dati, la stima del ciclo a 52 settimane usa quasi tutto il dataset")
print("per stimare un solo periodo stagionale → forza moderata attesa, non sottostimare questo limite.")

- **Trend:** leggera diminuzione nel periodo 2020-2021, con media non costante nel tempo
- **Stagionalità:** pattern periodico presente con forza moderata (~0.55); la periodicità è plausibile ma stimata su un solo ciclo completo
- **Residui:** oscillano intorno allo zero senza pattern sistematici evidenti

**Conclusione:** la serie presenta sia trend che stagionalità, quindi non è stazionaria in senso stretto. È necessario verificare con il test ADF e valutare l'entità della differenziazione necessaria.

## 6. Test di Stazionarietà (ADF)

Il test Augmented Dickey-Fuller (ADF) verifica l'ipotesi nulla che la serie abbia una radice unitaria (non stazionaria).  
**Se p < 0.05** → rifiuto H₀ → serie **stazionaria**.

In [ ]:
print("="*80)
print("FASE 6: TEST DI STAZIONARIETÀ (ADF)")
print("="*80)

def adf_test_report(series, title):
    """Esegue ADF test e stampa risultati formattati. Restituisce il risultato."""
    result = adfuller(series.dropna(), autolag='AIC')
    print(f"{'─'*80}")
    print(f"TEST ADF: {title}")
    print(f"{'─'*80}")
    print(f"ADF Statistic:        {result[0]:.6f}")
    print(f"p-value:              {result[1]:.6f}")
    print(f"Lags Used:            {result[2]}")
    print(f"Critical Value (1%):  {result[4]['1%']:.6f}")
    print(f"Critical Value (5%):  {result[4]['5%']:.6f}")
    if result[1] < 0.05:
        print("✓ Serie STAZIONARIA (p < 0.05) — H0 rifiutata")
    else:
        print("✗ Serie NON STAZIONARIA (p >= 0.05) — differenziazione necessaria")
    print()
    return result

print("\n1. SERIE ORIGINALE\n")
adf_original = adf_test_report(df_weekly['respiratory_admissions'], 'Ricoveri Settimanali (Originale)')

print("\n2. SERIE DIFFERENZIATA (d=1)\n")
diff_series = df_weekly['respiratory_admissions'].diff().dropna()
adf_diff = adf_test_report(diff_series, 'Ricoveri Settimanali (Differenziata d=1)')

print("\n3. SERIE DIFFERENZIATA STAGIONALMENTE (D=1, lag=52)\n")
seasonal_diff = df_weekly['respiratory_admissions'].diff(52).dropna()
adf_seasonal = adf_test_report(seasonal_diff, 'Ricoveri Settimanali (Diff. Stagionale D=1)')

print("\n4. SERIE COMPLETAMENTE DIFFERENZIATA (d=1 + D=1)\n")
full_diff = df_weekly['respiratory_admissions'].diff().diff(52).dropna()
adf_full = adf_test_report(full_diff, 'Ricoveri Settimanali (d=1 + D=1)')

print("="*80)
print("RIEPILOGO STAZIONARIETÀ")
print("="*80)
for label, result in [
    ('Originale    ', adf_original),
    ('d=1          ', adf_diff),
    ('D=1          ', adf_seasonal),
    ('d=1 + D=1    ', adf_full),
]:
    status = '✓ STAZIONARIA' if result[1] < 0.05 else '✗ NON STAZIONARIA'
    print(f"  {label}  p-value={result[1]:.6f}  {status}")

print()
if adf_original[1] < 0.05:
    print("⚠ La serie originale è GIÀ stazionaria (p ≈ 0.000013).")
    print("  → auto_arima con d=None sceglierà d=1 (AIC migliore) ma D=0,")
    print("    perché D=1 con s=52 su soli 84 sett. di training causa non-convergenza.")
    print("  → Il modello ottimale atteso è ARIMA(2,1,0) con D=0.")

## 7. Analisi ACF/PACF

L'**ACF** mostra la correlazione della serie con i suoi valori passati a diversi lag.  
La **PACF** isola la correlazione diretta, eliminando l'effetto dei lag intermedi.  

Si analizzano tre versioni della serie per guidare la scelta dei parametri p, q, P, Q del modello SARIMA.

In [ ]:
print("="*80)
print("FASE 7: ANALISI ACF/PACF — Serie Originale")
print("="*80)

fig, axes = plt.subplots(1, 2, figsize=(20, 6), dpi=100)
plot_acf(df_weekly['respiratory_admissions'].dropna(), lags=52, ax=axes[0])
axes[0].set_title('ACF – Serie Originale', fontsize=14, fontweight='bold')
plot_pacf(df_weekly['respiratory_admissions'].dropna(), lags=52, ax=axes[1])
axes[1].set_title('PACF – Serie Originale', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

- **ACF:** nessun pattern sinusoidale né picchi ricorrenti al lag 52 — coerente con la stazionarietà già rilevata dall'ADF. Quasi tutti i lag cadono dentro l'intervallo di confidenza.
- **PACF:** i lag 1 e 2 mostrano picchi significativi con taglio netto verso lag 3 → suggerisce una componente **AR(2)**. Questo guida auto_arima verso p=2.
- **Interpretazione:** la struttura AR(2) è il segnale principale. L'assenza di picchi stagionali chiari nell'ACF conferma che D=0 è la scelta corretta con questo numero di osservazioni.

In [ ]:
print("FASE 7b: ACF/PACF — Serie Differenziata (d=1)")

# FIX: max_lags calcolato dinamicamente
n_d1 = len(diff_series.dropna())
max_lags_d1 = min(52, n_d1 // 2 - 1)
print(f"Osservazioni dopo d=1: {n_d1} → max lags calcolabili: {max_lags_d1}")

fig, axes = plt.subplots(1, 2, figsize=(20, 6), dpi=100)
plot_acf(diff_series.dropna(), lags=max_lags_d1, ax=axes[0])
axes[0].set_title('ACF – Serie Differenziata (d=1)', fontsize=14, fontweight='bold')
plot_pacf(diff_series.dropna(), lags=max_lags_d1, ax=axes[1])
axes[1].set_title('PACF – Serie Differenziata (d=1)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
print("FASE 7c: ACF/PACF — Serie Completamente Differenziata (d=1 + D=1)")

n_full = len(full_diff.dropna())
max_lags_full = n_full // 2 - 1   # FIX: limite obbligatorio per PACF
print(f"Osservazioni dopo d=1+D=1: {n_full} (originale: {len(df_weekly)})")
print(f"Max lags PACF: {max_lags_full}  (limite = n_obs // 2 - 1)")
print("⚠ La doppia differenziazione consuma 53 osservazioni → conferma che D=1")
print("  con s=52 su ~105 settimane è eccessivo per la stima del modello.")

fig, axes = plt.subplots(1, 2, figsize=(20, 6), dpi=100)
plot_acf(full_diff.dropna(), lags=max_lags_full, ax=axes[0])
axes[0].set_title(f'ACF – d=1+D=1  (n={n_full}, max lag={max_lags_full})',
                  fontsize=14, fontweight='bold')
plot_pacf(full_diff.dropna(), lags=max_lags_full, ax=axes[1])
axes[1].set_title(f'PACF – d=1+D=1  (n={n_full}, max lag={max_lags_full})',
                  fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()
print("Se quasi tutto rientra nell'intervallo di confidenza → struttura AR/MA assente nei residui.")

## 8. Selezione Parametri con `auto_arima`

Con `d=None` e `D=None`, `auto_arima` usa internamente i test KPSS/ADF per scegliere la differenziazione ottimale, evitando over-differenziazione. La ricerca stepwise minimizza l'AIC.

**Risultato atteso:** `ARIMA(2,1,0)` con `D=0` — coerente con la PACF (picchi ai lag 1-2) e il numero limitato di osservazioni.

In [ ]:
print("="*80)
print("FASE 8: RICERCA PARAMETRI OTTIMALI (AUTO_ARIMA)")
print("="*80)
print("d=None, D=None → auto_arima determina la differenziazione ottimale tramite test statistici")
print("Ricerca in corso (può richiedere alcuni minuti)...\n")

sarima_auto = pm.auto_arima(
    df_weekly['respiratory_admissions'],
    seasonal=True,
    m=52,
    d=None,   # FIX: lascia decidere al test KPSS/ADF interno
    D=None,   # FIX: lascia decidere al test interno
    start_p=0, start_q=0,
    start_P=0, start_Q=0,
    max_p=2, max_q=2,
    max_P=2, max_Q=2,
    trace=True,
    error_action='ignore',
    suppress_warnings=True,
    stepwise=True,
    information_criterion='aic'
)

print("\n" + "─"*80)
print("MODELLO OTTIMALE TROVATO")
print("─"*80)
print(sarima_auto.summary())

best_order    = sarima_auto.order
best_seasonal = sarima_auto.seasonal_order
print(f"\nParametri ottimali: SARIMA{best_order} × {best_seasonal}")
print(f"AIC: {sarima_auto.aic():.3f}")

In [ ]:
sarima_auto.plot_diagnostics(figsize=(20, 12))
plt.suptitle('Diagnostica Modello SARIMA (auto_arima)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 9. Train/Test Split (80/20)

Divisione cronologica su `df_weekly` (dati settimanali aggregati).  
**IMPORTANTE:** si usa `df_weekly`, **non** `df_east` — il dataset grezzo giornaliero ha 635 osservazioni non aggregate, il che produrrebbe training su dati incoerenti con i modelli SARIMA settimanali.

In [ ]:
print("="*80)
print("FASE 9: TRAIN/TEST SPLIT")
print("="*80)

# FIX: split su df_weekly (settimanale), NON su df_east (giornaliero)
train_size = int(len(df_weekly) * 0.8)
train_w = df_weekly.iloc[:train_size].copy()
test_w  = df_weekly.iloc[train_size:].copy()

print(f"Training set: {train_w.index[0].date()} → {train_w.index[-1].date()} ({len(train_w)} settimane)")
print(f"Test set:     {test_w.index[0].date()} → {test_w.index[-1].date()} ({len(test_w)} settimane)")
print(f"Split:        {len(train_w)/len(df_weekly)*100:.1f}% / {len(test_w)/len(df_weekly)*100:.1f}%")

fig, ax = plt.subplots(figsize=(16, 5))
ax.plot(train_w.index, train_w['respiratory_admissions'],
        label=f'Training ({len(train_w)} sett.)', color='steelblue')
ax.plot(test_w.index, test_w['respiratory_admissions'],
        label=f'Test ({len(test_w)} sett.)', color='black')
ax.axvline(train_w.index[-1], color='red', linestyle='--', alpha=0.7, label='Punto di split')
ax.set_title('Train/Test Split – df_weekly', fontsize=13, fontweight='bold')
ax.set_ylabel('Ricoveri Settimanali')
ax.legend()
ax.grid(True, linestyle='--', alpha=0.4)
plt.tight_layout()
plt.show()

## 10. Fit dei Modelli

Vengono confrontati **tre modelli**, tutti addestrati su `train_w` (84 settimane settimanali aggregate):

| Modello | Specifiche | Note |
|---------|------------|------|
| **A** | SARIMA(auto_arima) con D=0 | Parametri scelti dai dati; convergenza garantita |
| **B** | SARIMA(2,1,0)×(1,1,0,52) | Best da auto_arima con D=1 forzato; possibile non-convergenza |
| **C** | SARIMAX(auto_arima) + PM2.5 + temperatura | Modello A con regressori esogeni ambientali |

> **Perché il Modello B può non convergere?** Con 84 settimane di training e `s=52`, la differenziazione stagionale `D=1` rimuove 52 osservazioni, lasciando solo ~32 gradi di libertà per stimare i parametri stagionali. Std err > 50 nelle componenti SAR segnalano non-convergenza della MLE.

In [ ]:
print("─"*80)
print(f"MODELLO A — SARIMA{best_order}×{best_seasonal} (da auto_arima, D=0)")
print("─"*80)

model_a = SARIMAX(
    train_w['respiratory_admissions'],
    order=best_order,
    seasonal_order=best_seasonal,
    trend='c' if best_order[1] == 0 else 'n'
)
fit_a = model_a.fit(disp=False)
print(fit_a.summary())

lb_a  = acorr_ljungbox(fit_a.resid.dropna(), lags=[10], return_df=True)
lbp_a = lb_a['lb_pvalue'].values[0]
print(f"\nLjung-Box test (lag=10): p-value = {lbp_a:.4f}")
print("✓ Residui non autocorrelati — modello adeguato" if lbp_a > 0.05
      else "⚠ Residui autocorrelati — struttura temporale residua")

forecast_a = fit_a.get_forecast(steps=len(test_w))
pred_a = forecast_a.predicted_mean
ci_a   = forecast_a.conf_int()
pred_a.index = test_w.index
ci_a.index   = test_w.index

mae_a  = mean_absolute_error(test_w['respiratory_admissions'], pred_a)
rmse_a = np.sqrt(mean_squared_error(test_w['respiratory_admissions'], pred_a))
r2_a   = r2_score(test_w['respiratory_admissions'], pred_a)
print(f"\n>>> Modello A — MAE: {mae_a:.2f}  |  RMSE: {rmse_a:.2f}  |  R²: {r2_a:.3f}")

In [ ]:
print("─"*80)
print("MODELLO B — SARIMA(2,1,0)×(1,1,0,52)  [best da auto_arima con D=1 forzato]")
print("NOTA: con 84 sett. di training, D=1 a s=52 lascia pochi gradi di libertà.")
print("Std err elevati nei parametri SAR segnalano non-convergenza MLE.")
print("─"*80)

# FIX: specifica (2,1,0)×(1,1,0,52) — il vero best trovato da auto_arima con D=1
model_b = SARIMAX(
    train_w['respiratory_admissions'],
    order=(2, 1, 0),
    seasonal_order=(1, 1, 0, 52),
    trend='n'
)
fit_b = model_b.fit(disp=False, maxiter=500)
print(fit_b.summary())

# Avviso automatico su non-convergenza
print("\n--- Verifica convergenza ---")
for param, se in zip(fit_b.params.index, fit_b.bse):
    if se > 50:
        print(f"⚠ {param}: std err = {se:.1f} — segnale di non-convergenza")

lb_b  = acorr_ljungbox(fit_b.resid.dropna(), lags=[10], return_df=True)
lbp_b = lb_b['lb_pvalue'].values[0]
print(f"\nLjung-Box test (lag=10): p-value = {lbp_b:.4f}")
print("✓ Residui non autocorrelati" if lbp_b > 0.05
      else "⚠ Residui autocorrelati — il modello non cattura tutta la struttura temporale")

forecast_b = fit_b.get_forecast(steps=len(test_w))
pred_b = forecast_b.predicted_mean
ci_b   = forecast_b.conf_int()
pred_b.index = test_w.index
ci_b.index   = test_w.index

mae_b  = mean_absolute_error(test_w['respiratory_admissions'], pred_b)
rmse_b = np.sqrt(mean_squared_error(test_w['respiratory_admissions'], pred_b))
r2_b   = r2_score(test_w['respiratory_admissions'], pred_b)
print(f"\n>>> Modello B — MAE: {mae_b:.2f}  |  RMSE: {rmse_b:.2f}  |  R²: {r2_b:.3f}")

In [ ]:
print("─"*80)
print(f"MODELLO C — SARIMAX{best_order}×{best_seasonal} con PM2.5 + temperatura")
print("─"*80)

exog_vars = ['PM2.5', 'temperature']
train_X = train_w[exog_vars]
test_X  = test_w[exog_vars]

model_c = SARIMAX(
    train_w['respiratory_admissions'],
    exog=train_X,
    order=best_order,
    seasonal_order=best_seasonal,
    trend='c' if best_order[1] == 0 else 'n'
)
fit_c = model_c.fit(disp=False)
print(fit_c.summary())

# Significatività regressori esogeni
print("\n--- Significatività variabili esogene ---")
for var in exog_vars:
    pval = fit_c.pvalues[var]
    sig = '✓ significativa (p < 0.05)' if pval < 0.05 else f'✗ NON significativa (p={pval:.3f}) — valutare rimozione'
    print(f"  {var}: p-value = {pval:.4f}  →  {sig}")

lb_c  = acorr_ljungbox(fit_c.resid.dropna(), lags=[10], return_df=True)
lbp_c = lb_c['lb_pvalue'].values[0]
print(f"\nLjung-Box test (lag=10): p-value = {lbp_c:.4f}")
print("✓ Residui non autocorrelati" if lbp_c > 0.05 else "⚠ Residui autocorrelati")

forecast_c = fit_c.get_forecast(steps=len(test_w), exog=test_X)
pred_c = forecast_c.predicted_mean
ci_c   = forecast_c.conf_int()
pred_c.index = test_w.index
ci_c.index   = test_w.index

mae_c  = mean_absolute_error(test_w['respiratory_admissions'], pred_c)
rmse_c = np.sqrt(mean_squared_error(test_w['respiratory_admissions'], pred_c))
r2_c   = r2_score(test_w['respiratory_admissions'], pred_c)
print(f"\n>>> Modello C — MAE: {mae_c:.2f}  |  RMSE: {rmse_c:.2f}  |  R²: {r2_c:.3f}")

## 11. Confronto Visivo delle Previsioni

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(16, 16), sharex=True)

configs = [
    (pred_a, ci_a, f'Modello A — SARIMA{best_order}×{best_seasonal} (auto_arima, D=0)',
     'darkorange', mae_a, rmse_a, r2_a),
    (pred_b, ci_b,  'Modello B — SARIMA(2,1,0)×(1,1,0,52) con D=1',
     'crimson', mae_b, rmse_b, r2_b),
    (pred_c, ci_c, f'Modello C — SARIMAX{best_order}×{best_seasonal} + PM2.5 + Temperatura',
     'purple', mae_c, rmse_c, r2_c),
]

for ax, (pred, ci, label, color, mae, rmse, r2) in zip(axes, configs):
    ax.plot(train_w.index, train_w['respiratory_admissions'],
            label='Training', color='steelblue', alpha=0.5, linewidth=1.5)
    ax.plot(test_w.index, test_w['respiratory_admissions'],
            label='Test (Actual)', color='black', linewidth=2.5)
    ax.plot(test_w.index, pred,
            label='Previsione', color=color, linewidth=2.5)
    ax.fill_between(test_w.index, ci.iloc[:, 0], ci.iloc[:, 1],
                    color=color, alpha=0.15, label='95% CI')
    ax.set_title(f'{label}\nMAE={mae:.1f}  |  RMSE={rmse:.1f}  |  R²={r2:.3f}',
                 fontsize=12, fontweight='bold')
    ax.set_ylabel('Ricoveri Settimanali', fontsize=11)
    ax.legend(loc='upper left', fontsize=10)
    ax.grid(True, linestyle='--', alpha=0.4)

axes[-1].set_xlabel('Data', fontsize=12)
plt.tight_layout()
plt.show()

## 12. Tabella Comparativa Finale

In [ ]:
print("="*80)
print("TABELLA COMPARATIVA FINALE — Metriche sul Test Set")
print("="*80)

results = pd.DataFrame({
    f'A – SARIMA{best_order}×{best_seasonal}': {
        'MAE': round(mae_a, 2),
        'RMSE': round(rmse_a, 2),
        'R²': round(r2_a, 3),
        'AIC (train)': round(fit_a.aic, 1),
        'Ljung-Box p': round(lbp_a, 4),
        'Convergenza': 'OK',
        'Esogene': 'No'
    },
    'B – SARIMA(2,1,0)×(1,1,0,52)': {
        'MAE': round(mae_b, 2),
        'RMSE': round(rmse_b, 2),
        'R²': round(r2_b, 3),
        'AIC (train)': round(fit_b.aic, 1),
        'Ljung-Box p': round(lbp_b, 4),
        'Convergenza': 'Verificare std err SAR',
        'Esogene': 'No'
    },
    f'C – SARIMAX{best_order} + PM2.5 + Temp': {
        'MAE': round(mae_c, 2),
        'RMSE': round(rmse_c, 2),
        'R²': round(r2_c, 3),
        'AIC (train)': round(fit_c.aic, 1),
        'Ljung-Box p': round(lbp_c, 4),
        'Convergenza': 'OK',
        'Esogene': 'PM2.5, temperatura'
    },
}).T

print(results.to_string())

best_model = results['MAE'].astype(float).idxmin()
print(f"\n→ Modello con MAE minore sul test set: {best_model}")
print(f"  MAE={results.loc[best_model,'MAE']}  RMSE={results.loc[best_model,'RMSE']}  R²={results.loc[best_model,'R²']}")

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

model_labels = ['A (auto)', 'B (D=1)', 'C (SARIMAX)']
colors = ['darkorange', 'crimson', 'purple']

for ax, metric, vals in zip(
    axes,
    ['MAE', 'RMSE', 'R²'],
    [[mae_a, mae_b, mae_c], [rmse_a, rmse_b, rmse_c], [r2_a, r2_b, r2_c]]):
    bars = ax.bar(model_labels, vals, color=colors, edgecolor='black', alpha=0.85, width=0.5)
    ax.set_title(metric, fontsize=13, fontweight='bold')
    ax.set_ylabel(metric)
    for bar, val in zip(bars, vals):
        ax.text(bar.get_x() + bar.get_width()/2., bar.get_height() * 1.015,
                f'{val:.2f}', ha='center', va='bottom', fontsize=11, fontweight='bold')
    ax.grid(True, axis='y', alpha=0.3)
    ax.set_ylim(bottom=0)

plt.suptitle('Confronto Metriche — Test Set', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 13. Conclusioni

### Risultati principali

**Test ADF:** la serie originale settimanale è già stazionaria (p ≈ 0.000013). `auto_arima` con `d=None, D=None` sceglie `d=1` per robustezza (AIC migliore) e `D=0`, perché con 84 settimane di training la differenziazione stagionale a `s=52` non è stimabile in modo affidabile.

**Modello A — SARIMA(2,1,0)×(0,0,0,52):** modello più robusto. La struttura AR(2) è coerente con i picchi nella PACF ai lag 1-2. I coefficienti AR sono entrambi significativi; i residui non mostrano autocorrelazione (Ljung-Box p > 0.05).

**Modello B — SARIMA(2,1,0)×(1,1,0,52):** la componente stagionale con D=1 produce std err molto elevati nei parametri SAR, segnale di non-convergenza MLE causato dall'insufficiente numero di osservazioni nel training set. Le metriche sul test set vanno interpretate con cautela.

**Modello C — SARIMAX + PM2.5 + temperatura:** aggiunge i regressori esogeni al Modello A. La temperatura risulta sistematicamente non significativa (p > 0.4). Il PM2.5 può risultare significativo a seconda della specifica, ma il contributo alle metriche sul test set è modesto.

### Limitazioni strutturali del dataset

- **Dataset troppo corto:** ~2 anni di dati (105 settimane totali, 84 per il training) sono insufficienti per stimare affidabilmente una stagionalità a periodo 52. Servirebbero almeno 3-4 anni.
- **Singola regione:** l'analisi è limitata alla regione East. Un confronto multi-regionale permetterebbe conclusioni più robuste.
- **Relazione PM2.5 debole:** l'effetto del PM2.5 sui ricoveri è presente ma debole e dipende fortemente dalla specifica del modello e dal lag considerato. Non è sufficiente per affermare una relazione causale.